In [ ]:
# from https://colab.research.google.com/drive/1Z6VIRZ_sX314hyev3Gm5gBqvm1wQVo-a#scrollTo=RtMcXr3o6gxN

In [1]:
from IPython.display import clear_output
!pip install ffmpeg-python
!pip install SpeechRecognition
!apt install libasound2-dev portaudio19-dev libportaudio2 libportaudiocpp0 ffmpeg
!pip install pyaudio
import scipy
clear_output()

In [2]:
# @title
"""
To write this piece of code I took inspiration/code from a lot of places.
It was late night, so I'm not sure how much I created or just copied o.O
Here are some of the possible references:
https://blog.addpipe.com/recording-audio-in-the-browser-using-pure-html5-and-minimal-javascript/
https://stackoverflow.com/a/18650249
https://hacks.mozilla.org/2014/06/easy-audio-capture-with-the-mediarecorder-api/
https://air.ghost.io/recording-to-an-audio-file-using-html5-and-js/
https://stackoverflow.com/a/49019356
"""
from IPython.display import HTML, Audio
from google.colab.output import eval_js
from base64 import b64decode
import numpy as np
from scipy.io.wavfile import read as wav_read
import io
import ffmpeg

AUDIO_HTML = """
<script>
var my_div = document.createElement("DIV");
var my_p = document.createElement("P");
var my_btn = document.createElement("BUTTON");
var t = document.createTextNode("Press to start recording");

my_btn.appendChild(t);
//my_p.appendChild(my_btn);
my_div.appendChild(my_btn);
document.body.appendChild(my_div);

var base64data = 0;
var reader;
var recorder, gumStream;
var recordButton = my_btn;

var handleSuccess = function(stream) {
  gumStream = stream;
  var options = {
    //bitsPerSecond: 8000, //chrome seems to ignore, always 48k
    mimeType : 'audio/webm;codecs=opus'
    //mimeType : 'audio/webm;codecs=pcm'
  };
  //recorder = new MediaRecorder(stream, options);
  recorder = new MediaRecorder(stream);
  recorder.ondataavailable = function(e) {
    var url = URL.createObjectURL(e.data);
    var preview = document.createElement('audio');
    preview.controls = true;
    preview.src = url;
    document.body.appendChild(preview);

    reader = new FileReader();
    reader.readAsDataURL(e.data);
    reader.onloadend = function() {
      base64data = reader.result;
      //console.log("Inside FileReader:" + base64data);
    }
  };
  recorder.start();
  };

recordButton.innerText = "Recording... press to stop";

navigator.mediaDevices.getUserMedia({audio: true}).then(handleSuccess);


function toggleRecording() {
  if (recorder && recorder.state == "recording") {
      recorder.stop();
      gumStream.getAudioTracks()[0].stop();
      recordButton.innerText = "Saving the recording... pls wait!"
  }
}

// https://stackoverflow.com/a/951057
function sleep(ms) {
  return new Promise(resolve => setTimeout(resolve, ms));
}

var data = new Promise(resolve=>{
//recordButton.addEventListener("click", toggleRecording);
recordButton.onclick = ()=>{
toggleRecording()

sleep(2000).then(() => {
  // wait 2000ms for the data to be available...
  // ideally this should use something like await...
  //console.log("Inside data:" + base64data)
  resolve(base64data.toString())

});

}
});

</script>
"""

def get_audio():
  display(HTML(AUDIO_HTML))
  data = eval_js("data")
  binary = b64decode(data.split(',')[1])

  process = (ffmpeg
    .input('pipe:0')
    .output('pipe:1', format='wav')
    .run_async(pipe_stdin=True, pipe_stdout=True, pipe_stderr=True, quiet=True, overwrite_output=True)
  )
  output, err = process.communicate(input=binary)

  riff_chunk_size = len(output) - 8
  # Break up the chunk size into four bytes, held in b.
  q = riff_chunk_size
  b = []
  for i in range(4):
      q, r = divmod(q, 256)
      b.append(r)

  # Replace bytes 4:8 in proc.stdout with the actual size of the RIFF chunk.
  riff = output[:4] + bytes(b) + output[8:]

  sr, audio = wav_read(io.BytesIO(riff))

  return audio, sr

In [3]:
# @title
import speech_recognition as sr
import time
from os import path

class DoSpeech:

    # class to handle speech recognition by voice or file
    def __init__(self, __file__ = "", timeSlices = [],
                 service = "Google_Speech_Recognition", verbose = True):
        from os import path

        self.AUDIO_FILE = __file__
        self.verbose = verbose
        # list of time slices to process in milliseconds from beginning
        # processes whole input if empty
        self.timeSlices = timeSlices # [ [1560, 2560], ... ]
        self.service = service
        self.r = sr.Recognizer()
        self.output = []
        self.outputDict = []

    def msTime(self): # return current time in seconds
        return round(time.time() * 1000)

    def printMicrophoneDevices(self): # prints device numbers
        for index, name in enumerate(sr.Microphone.list_microphone_names()):
            print("Microphone with name \"{1}\" found for `Microphone(device_index={0})`".format(index, name))


    def recSpeech(self, audio, service,start= None, end=None):
        # recognize speech in audio using service (internal function)
        try:
            # for testing purposes, we're just using the default API key
            # to use another API key, use `r.recognize_google(audio, key="GOOGLE_SPEECH_RECOGNITION_API_KEY")`
            # instead of `r.recognize_google(audio)`
            if service == 'Google_Speech_Recognition':
                self.outputDict.append(self.r.recognize_google(audio, show_all = True))
                self.output.append([self.r.recognize_google(audio),start,end])
                if self.verbose:
                    print(f"{service} thinks you said:\n " + self.output[-1][0])
        except sr.UnknownValueError:
            print("Google Speech Recognition could not understand audio")
        except sr.RequestError as e:
            print("Could not request results from Google Speech Recognition service; {0}".format(e))


    def recognizeSpeechFromFile(self, file, timeSlices = [[None,None]]):
        # process audio file from start to end point in seconds -->> [ [None, 5.0]...]
        self.AUDIO_FILE = file
        print("using: ", self.AUDIO_FILE)
        for each in timeSlices:
            print(f"processing {each} time slice...")
            start = each[0]
            end = each[1]
            with sr.AudioFile(self.AUDIO_FILE) as source:
                if start == None and end == None:
                    audio = self.r.record(source) # read entire file
                elif end == None:
                    audio = self.r.record(source, offset = start) # read to the end
                else:
                    audio = self.r.record(source, offset = start, duration = end-start) # read slice

            if self.verbose:
                pass
                #print(self.msTime())
            self.recSpeech(audio, self.service,start,end)
            if self.verbose:
                pass
                #print(self.msTime())
            print()
        print()

    def recognizeSpeechFromMic(self):
        # process microphone input
        # obtain audio from the microphone
        # sr.Microphone(device_index=0) for specific device (find index above)
        with sr.Microphone() as source:
            print("Say something!")
            audio = self.r.listen(source)

        if self.verbose:
            pass
            #print(self.msTime())
        self.recSpeech(audio, self.service)
        if self.verbose:
            pass
            #print(self.msTime())
        print()




In [4]:
start = input("Press enter, wait one second, and start speaking.")
audio, sr2 = get_audio()
scipy.io.wavfile.write('recording.wav', sr2, audio)

Press enter, wait one second, and start speaking.


In [5]:

y = DoSpeech()
y.recognizeSpeechFromFile("/content/recording.wav",timeSlices=[[1.6, 3.987664],[None,None]])
#y.recognizeSpeechFromMic()
print(y.output)
print()
print(y.outputDict)


using:  /content/recording.wav
processing [1.6, 3.987664] time slice...
Google_Speech_Recognition thinks you said:
 test test test test test test test

processing [None, None] time slice...
Google_Speech_Recognition thinks you said:
 test test test test test test test test test test test test test test test test test test test test test test test test test test test test test test hello hello hello hello hello


[['test test test test test test test', 1.6, 3.987664], ['test test test test test test test test test test test test test test test test test test test test test test test test test test test test test test hello hello hello hello hello', None, None]]

[{'alternative': [{'transcript': 'test test test test test test test', 'confidence': 0.85964876}, {'transcript': 'test test test test test test test test', 'confidence': 0.87009811}, {'transcript': 'test test test test test test', 'confidence': 0.84592795}, {'transcript': 'test test test test test test test test test', 'confiden